In [2]:
#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

# 시각화 OS별 한글폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 폰트 설정
elif platform.system() == 'Mac':
    plt.rcParams['font.family'] = 'AppleGothic'  # Mac 폰트 설정

Windows


In [3]:
# 아이템,챔프 정보 데이터셋 호출
# 구분 쉽게볼 용도
df_champ = pd.read_csv('TFT_Champion.csv')
df_item  = pd.read_csv('TFT_Item.csv')

#정식 제출용 코드
# df_champ = pd.read_csv('TFT_Champion_CurrentVersion.csv')
# df_item  = pd.read_csv('TFT_Item_CurrentVersion.csv')

In [ ]:
# 티어별 파일과 이름 매핑
# 구분 쉽게볼 용도
tier_files = {
    "Challenger"    : "TFT_0_CH.csv",
    "GrandMaster"   : "TFT_0_GM.csv",
    "Master"        : "TFT_0_MT.csv",
    "Diamond"       : "TFT_0_DI.csv",
    "Platinum"      : "TFT_0_PT.csv"
}

# 정식 제출용 코드 
# tier_files = {
#     "Challenger": "TFT_Challenger_MatchData.csv",
#     "GrandMaster": "TFT_GrandMaster_MatchData.csv",
#     "Master": "TFT_Master_MatchData.csv",
#     "Diamond": "TFT_Diamond_MatchData.csv",
#     "Platinum": "TFT_Platinum_MatchData.csv"
# 데이터프레임 병합
df_list = []
for tier, file in tier_files.items():
    df_raw = pd.read_csv(file, encoding='utf-8')
    df_raw["tier"] = tier
    print(f"[{tier}] 처리 전 데이터 크기: {df.shape}")
    df_list.append(df_raw)

# 통합
df_raw = pd.concat(df_list, ignore_index=True)

# ignore_index = 기존 인덱스 버리고 새 데이터에 0부터 순차적으로 인덱스 부여
print("="*60)
print("통합 후 데이터 크기:", df.shape)

[Challenger] 처리 전 데이터 크기: (79999, 9)
[GrandMaster] 처리 전 데이터 크기: (80000, 9)
[Master] 처리 전 데이터 크기: (79999, 9)
[Diamond] 처리 전 데이터 크기: (80000, 9)
[Platinum] 처리 전 데이터 크기: (80000, 9)
통합 후 데이터 크기: (399998, 9)


In [5]:
df.to_csv('TFT_Total_row.csv')

In [6]:
# 결측제거 적용 전 쉐이프 확인
print(f'결측 제거 전 : {df.shape}')

# 인덱스 수집
idx1 = df[df['Ranked'] == 0].index
idx2 = df[df['champion'] == '{}'].index
idx3 = df[df['combination'] == '{}'].index

# 모든 인덱스 통합 (중복 제거)
drop_idx = idx1.union(idx2).union(idx3)

# 한 번에 삭제
df = df.drop(index=drop_idx)

# 결과 확인
print(f'결측 제거 후 : {df.shape}')

결측 제거 전 : (399998, 9)
결측 제거 후 : (399484, 9)


In [7]:
import ast

# ✅ 문자열 → 딕셔너리 변환
for col in ['champion', 'combination']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# ✅ 챔피언 정렬 함수 (간결화)
def expand_champion_info(d, n=4):
    if not isinstance(d, dict):
        return {}

    champs = [
        {   'name': k,
            'star': int(v.get('star', 0)),
            'items': (v.get('items') or None),
            'item_count': len(v.get('items', []))
        }
        for k, v in d.items()
    ]
    # 아이템 수 ↓, star ↓
    s = sorted(champs, key=lambda x: (x['item_count'], x['star']), reverse=True)

    # 🔹 여기서부터 교차 순서로 삽입
    res = {}
    for i, c in enumerate(s[:n], start=1):
        res[f'Champion{i}'] = c.get('name')
        res[f'star{i}'] = int(c.get('star', 0)) if c.get('star') is not None else 0  # ⭐ 항상 정수
        res[f'items{i}']    = c.get('items')

    res['entry'] = [c['name'] for c in s]
    return res

# ✅ 조합 정렬 함수 (간결화)
def extract_combinations(d, n=3):
    if not isinstance(d, dict): return {}
    s = sorted(d.items(), key=lambda x: x[1], reverse=True)
    f = [f'{k} {int(v)}' for k, v in s[:n]]
    keys = [k for k, _ in s]
    return {'main_comb1': f[0] if len(f) > 0 else None,
            'main_comb2': f[1] if len(f) > 1 else None,
            'sub_comb': f[2] if len(f) > 2 else None,
            'sorted_keys': keys}

# ✅ 컬럼 확장 및 병합
expanded_champ = df['champion'].apply(expand_champion_info).apply(pd.Series)
expanded_comb = df['combination'].apply(extract_combinations).apply(pd.Series)
df = pd.concat([df, expanded_champ, expanded_comb], axis=1)

# ✅ combination 정렬 리스트 반영
df['combination'] = df['sorted_keys']

# ✅ combination 컬럼을 정렬된 key 리스트로 업데이트
df['combination'] = df['sorted_keys']

# ✅ entry 컬럼을 7번째 위치 & main_comb 컬럼을 9,10,11번째 위치에 삽입
entry_col = df.pop('entry')            # ✅ 문자열만 전달
df.insert(6, 'entry', entry_col)

for i, col in enumerate(['main_comb1', 'main_comb2', 'sub_comb']):
    col_data = df.pop(col)
    df.insert(8 + i, col, col_data)

# ✅ champion, sorted_keys 컬럼 제거
df = df.drop(columns=['champion','sorted_keys'])

In [8]:
df.to_csv('TFT_Match_info.csv')

In [9]:
df.head()

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,entry,combination,main_comb1,main_comb2,sub_comb,tier,Champion1,star1,items1,Champion2,star2,items2,Champion3,star3,items3,Champion4,star4,items4
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"[XinZhao, Rakan, JarvanIV, Sona, Lulu, Neeko, Karma, Soraka]","[Protector, Set3_Mystic, Set3_Celestial, DarkStar, StarGuardian, Rebel, TemplateTrait]",Protector 4,Set3_Mystic 4,Set3_Celestial 3,Challenger,XinZhao,3,"[69, 25, 25]",Rakan,3.0,"[37, 69]",JarvanIV,3.0,[27],Sona,3.0,[46]
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"[Yasuo, Kayle, MissFortune, AurelionSol, Malphite, Sona, Jinx, Soraka, MasterYi]","[Rebel, Set3_Blademaster, Blaster, Set3_Mystic, Valkyrie, Mercenary, Set3_Brawler, StarGuardian, Starship]",Rebel 6,Set3_Blademaster 3,Blaster 2,Challenger,Yasuo,2,"[99, 22, 23]",Kayle,2.0,"[26, 49, 49]",MissFortune,1.0,"[99, 26, 12]",AurelionSol,2.0,"[37, 66]"
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"[KaiSa, Shaco, Rumble, Annie, Ekko, Karma, Lux, Fizz]","[Infiltrator, DarkStar, MechPilot, Set3_Sorcerer, Cybernetic, Demolitionist, Set3_Mystic, Valkyrie]",Infiltrator 4,DarkStar 3,MechPilot 3,Challenger,KaiSa,2,"[99, 2, 23]",Shaco,2.0,"[16, 19, 55]",Rumble,3.0,"[36, 45]",Annie,2.0,[37]
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"[Shaco, Fizz, KaiSa, Gangplank, Annie, Rumble, MissFortune, Ekko]","[Infiltrator, MechPilot, Demolitionist, Mercenary, Valkyrie, Blaster, Cybernetic, DarkStar, Set3_Sorcerer, SpacePirate, TemplateTrait]",Infiltrator 4,MechPilot 3,Demolitionist 2,Challenger,Shaco,2,"[15, 16, 14]",Fizz,2.0,"[99, 46, 55]",KaiSa,2.0,"[44, 37]",Gangplank,2.0,"[37, 36]"
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"[AurelionSol, Gangplank, MissFortune, MasterYi, Lulu, Ziggs, Yasuo, Sona, Jinx]","[Rebel, Blaster, Demolitionist, Mercenary, Set3_Blademaster, Set3_Mystic, Set3_Celestial, SpacePirate, Starship, Valkyrie]",Rebel 6,Blaster 2,Demolitionist 2,Challenger,AurelionSol,2,"[46, 36, 44]",Gangplank,1.0,"[49, 33, 15]",MissFortune,2.0,"[12, 79]",MasterYi,2.0,[5]
